In [1]:
library(tidyverse)

players_url <- "https://raw.githubusercontent.com/cindylemon/plaicraft-individual-project/refs/heads/main/players.csv"
players <- read_csv(players_url)
head(players)

sessions_url <- "https://raw.githubusercontent.com/cindylemon/plaicraft-individual-project/refs/heads/main/sessions.csv"
sessions <- read_csv(sessions_url)
head(sessions)

ERROR: Error in read_csv(players_url): could not find function "read_csv"


How are you going to process the data to apply it
   to the model? (ex. how many splits, what proportions 
                  will you use for the splits, at which
                  stage will you split, will there be a 
                  validation set? Will you use cross
                  validation? 

- 5 splits will be used
- 75% of the data will be used for training, 25% for testing
-5 fold cross-validation will be used with the minumum RMSPE K value
- We first standardize the data in the recipe
- Then we make the spec
- then we use 5 fold cross validation
  

Broad Question: Question 2 (We would like to know which "kinds"
                            of players are most likely to contribute
                            a large amount of data so that we can target
                            those players in our recruiting efforts)

Specific Question: What gender and age group plays the longest hours? 

How the data will address the question of interest 
(The plan for wrangling to apply a predictive method 
 from class):

The data will find the gender and age groups that play the most hours, 
so that they can be recruited as they contribute a large amount of data. 

The plan is to use multivariable KNN regression in order to predict
the hours a participant of a certain gender and age
group will play. 

The players data set will be used.